In [1]:
import numpy as np
import cv2

webcam = cv2.VideoCapture(0)

while True:
    _, imageFrame = webcam.read()

    hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)

    # Define ranges for colors
    colors = {
        "Red": ([136, 87, 111], [180, 255, 255], (0, 0, 255)),
        "Green": ([25, 52, 72], [102, 255, 255], (0, 255, 0)),
        "Blue": ([94, 80, 2], [120, 255, 255], (255, 0, 0)),
        "Yellow": ([20, 100, 100], [30, 255, 255], (0, 255, 255)),
        "Purple": ([125, 50, 40], [155, 255, 255], (128, 0, 128)),
        "Black": ([0, 0, 0], [180, 255, 30], (0, 0, 0)),
        "Pink": ([150, 50, 50], [180, 255, 255], (255, 192, 203)),
        "Orange": ([5, 100, 100], [15, 255, 255], (255, 165, 0)),
    }

    for color_name, (lower, upper, rectangle_color) in colors.items():
        color_lower = np.array(lower, np.uint8)
        color_upper = np.array(upper, np.uint8)

        # Create mask and apply dilation
        color_mask = cv2.inRange(hsvFrame, color_lower, color_upper)
        kernel = np.ones((5, 5), "uint8")
        color_mask = cv2.dilate(color_mask, kernel)

        # Find contours only if the image is not None
        if color_mask is not None:
            contours, hierarchy = cv2.findContours(color_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 300:
                    x, y, w, h = cv2.boundingRect(contour)
                    imageFrame = cv2.rectangle(imageFrame, (x, y), (x + w, y + h), rectangle_color, 2)
                    cv2.putText(imageFrame, f"{color_name} Colour", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, rectangle_color)

    cv2.imshow("Multiple Color Detection in Real-Time", imageFrame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        webcam.release()
        cv2.destroyAllWindows()
        break
